In [1]:
import pandas as pd
import os
import sys
import numpy as np
import scipy

# remove
from tqdm.notebook import tqdm

# helpers
sys.path.append("/workspaces/ProductionRecommender/prod-reco/src/prod_reco")
from commons.recommender_utils import RecommenderUtils

In [2]:
df_ratings = catalog.load("ratings")
df_items = catalog.load("items")

2022-03-25 16:49:22,531 - kedro.io.data_catalog - INFO - Loading data from `ratings` (CSVDataSet)...


/opt/conda/envs/dev/lib/python3.8/site-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


2022-03-25 16:49:29,262 - kedro.io.data_catalog - INFO - Loading data from `items` (CSVDataSet)...


In [3]:
# PARAMETERS
USER_ID = "userId"
ITEM_ID = "itemId"
RATING="rating"

item_min_bought = 8
user_min_bought = 8

## Data Engineering - Process Ratings

In [4]:
# ratings of 5 are 1, everything else is deleted
df_ratings = df_ratings[df_ratings[RATING] > 4]
# original ratings
# df_ratings[RATING] = 1

# create utils object
utils = RecommenderUtils(user_id = USER_ID, item_id = ITEM_ID, rating=RATING)

# threshold interactions
df_txn = RecommenderUtils.threshold_interactions_df(df_ratings, USER_ID, ITEM_ID, user_min_bought, item_min_bought)
num_ratings, sparsity = utils.print_ratings_shape(df_txn)

# to sparse matrix
interactions, rid_to_idx, idx_to_rid, cid_to_idx, idx_to_cid = RecommenderUtils.df_to_matrix(df_txn, USER_ID, ITEM_ID, interaction_var=RATING)

num_users = interactions.shape[0]
num_items = interactions.shape[1]

Starting interactions info
Number of rows: 7212
Number of cols: 6432
Sparsity: 0.460%
Ending interactions info
Number of rows: 5387
Number of columns: 2620
Sparsity: 1.384%
Number of users: 5387
Number of items: 2620
Number of rows: (195359, 4)
Sparsity: 0.013841563730609597


In [5]:
# reorder and filter df_items
df_items = df_items.set_index(ITEM_ID).loc[cid_to_idx]

# get metadata tags
df_items_feats = df_items["tags"]

# get movie names
df_item_names = df_items["movieName"]
idx_to_names = {cid_to_idx[k]:v for k,v in df_item_names.to_dict().items()}

# Random Users

In [14]:
user_id = 2241
list_movies = df_txn[df_txn[USER_ID] == user_id][ITEM_ID].tolist()
list_ratings = df_txn[df_txn[USER_ID] == user_id][RATING].tolist()
[(idx_to_names[cid_to_idx[v]],r) for v,r in zip(list_movies, list_ratings)]

[('Hate (Haine, La) (1995)', 5.0),
 ('Taxi Driver (1976)', 5.0),
 ('Belle de jour (1967)', 5.0),
 ('Crumb (1994)', 5.0),
 ('Living in Oblivion (1995)', 5.0),
 ('Heavenly Creatures (1994)', 5.0),
 ('Swimming with Sharks (1995)', 5.0),
 ('Wonderful, Horrible Life of Leni Riefenstahl, The (Macht der Bilder: Leni Riefenstahl, Die) (1993)',
  5.0),
 ('Naked (1993)', 5.0),
 ('Short Cuts (1993)', 5.0),
 ('Welcome to the Dollhouse (1995)', 5.0),
 ('Fargo (1996)', 5.0),
 ('Trainspotting (1996)', 5.0),
 ('Godfather, The (1972)', 5.0),
 ('Casablanca (1942)', 5.0),
 ('Citizen Kane (1941)', 5.0),
 ('Bonnie and Clyde (1967)', 5.0),
 ('Reservoir Dogs (1992)', 5.0),
 ('Streetcar Named Desire, A (1951)', 5.0),
 ("One Flew Over the Cuckoo's Nest (1975)", 5.0),
 ('To Kill a Mockingbird (1962)', 5.0),
 ('Apocalypse Now (1979)', 5.0),
 ('Goodfellas (1990)', 5.0),
 ('Godfather: Part II, The (1974)', 5.0),
 ('Once Upon a Time in America (1984)', 5.0),
 ('Raging Bull (1980)', 5.0),
 ('Seventh Seal, The (Sjund

In [6]:
df_txn[USER_ID].sample(10)

286146    2241
781431    5917
456232    3468
109806     836
310729    2448
494717    3784
30174      226
709679    5411
743346    5669
23320      190
Name: userId, dtype: int64

In [23]:
user_id = 190
list_movies = df_txn[df_txn[USER_ID] == user_id][ITEM_ID].tolist()
list_ratings = df_txn[df_txn[USER_ID] == user_id][RATING].tolist()
[(idx_to_names[cid_to_idx[v]],r) for v,r in zip(list_movies, list_ratings)]

[('Toy Story (1995)', 4.5),
 ('Apollo 13 (1995)', 4.5),
 ('Star Wars: Episode IV - A New Hope (a.k.a. Star Wars) (1977)', 5.0),
 ('Star Trek: Generations (1994)', 4.5),
 ('Jurassic Park (1993)', 4.5),
 ('Terminator 2: Judgment Day (1991)', 5.0),
 ('Monty Python and the Holy Grail (1975)', 4.5),
 ('Star Wars: Episode V - The Empire Strikes Back (1980)', 5.0),
 ('Princess Bride, The (1987)', 5.0),
 ('Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)',
  4.5),
 ('Terminator, The (1984)', 4.5),
 ('Star Trek II: The Wrath of Khan (1982)', 4.5),
 ('Labyrinth (1986)', 5.0),
 ('Return to Oz (1985)', 4.5),
 ('Tron (1982)', 4.5),
 ('Willow (1988)', 4.5),
 ('Matrix, The (1999)', 5.0),
 ('Iron Giant, The (1999)', 5.0),
 ('Princess Mononoke (Mononoke-hime) (1997)', 5.0),
 ('Galaxy Quest (1999)', 4.5),
 ('X-Men (2000)', 5.0),
 ('Cast Away (2000)', 4.5),
 ('Shrek (2001)', 4.5),
 ("Harry Potter and the Sorcerer's Stone (a.k.a. Harry Potter and the Philosopher's Stone) (200

In [26]:
[(v, r) for v,r in zip(list_movies, list_ratings)]

[(1, 1),
 (5, 1),
 (27, 1),
 (31, 1),
 (39, 1),
 (47, 1),
 (74, 1),
 (104, 1),
 (158, 1),
 (215, 1),
 (261, 1),
 (262, 1),
 (339, 1),
 (367, 1),
 (372, 1),
 (374, 1),
 (377, 1),
 (381, 1),
 (454, 1),
 (480, 1),
 (507, 1),
 (524, 1),
 (595, 1),
 (662, 1),
 (724, 1),
 (765, 1),
 (778, 1),
 (830, 1),
 (1013, 1),
 (1020, 1),
 (1022, 1),
 (1059, 1),
 (1092, 1),
 (1097, 1),
 (1196, 1),
 (1246, 1),
 (1249, 1),
 (1259, 1),
 (1359, 1),
 (1380, 1),
 (1466, 1),
 (1552, 1),
 (1569, 1),
 (1573, 1),
 (1625, 1),
 (1645, 1),
 (1658, 1),
 (1777, 1),
 (1821, 1),
 (1895, 1),
 (2059, 1),
 (2081, 1),
 (2082, 1),
 (2085, 1),
 (2125, 1),
 (2272, 1),
 (2294, 1),
 (2310, 1),
 (2324, 1),
 (2329, 1),
 (2485, 1),
 (2541, 1),
 (2563, 1),
 (2572, 1),
 (2580, 1),
 (2581, 1),
 (2605, 1),
 (2676, 1),
 (2694, 1),
 (2707, 1),
 (2724, 1),
 (2771, 1),
 (2797, 1),
 (2908, 1),
 (3046, 1),
 (3083, 1),
 (3114, 1),
 (3147, 1),
 (3155, 1),
 (3173, 1),
 (3249, 1),
 (3250, 1),
 (3285, 1),
 (3409, 1),
 (3882, 1),
 (3916, 1),
 (394